In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [85]:
df = pd.read_csv("C:/Users/anies/Downloads/Bank_Personal_Loan_Modelling.csv")

In [86]:
df.drop(['ID'],inplace=True,axis=1)

In [87]:
df['Age'] = pd.cut(df['Age'],bins=[23,30,45,67],labels=['Young','Adult','Old'])

In [88]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Age'] = le.fit_transform(df['Age'])

In [89]:
col = ['Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg', 'Education', 
       'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']

In [90]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [91]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25)

In [94]:
from keras.models import Sequential
from keras.layers import Dense

keras = Sequential()
keras.add(Dense(12,input_dim=12,activation='relu'))
keras.add(Dense(8,activation='relu'))
keras.add(Dense(6,activation='relu'))

In [96]:
keras.add(Dense(1,activation='sigmoid'))

In [98]:
keras.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [99]:
keras.fit(x_train,y_train,epochs=20,batch_size=25)

Epoch 1/20
150/150 [==============================] - 3s 3ms/step - loss: 0.6169 - accuracy: 0.7112
Epoch 2/20
150/150 [==============================] - 0s 3ms/step - loss: 0.6071 - accuracy: 0.7112
Epoch 3/20
150/150 [==============================] - 0s 3ms/step - loss: 0.6032 - accuracy: 0.7112
Epoch 4/20
150/150 [==============================] - 0s 3ms/step - loss: 0.6018 - accuracy: 0.7112
Epoch 5/20
150/150 [==============================] - 1s 4ms/step - loss: 0.6013 - accuracy: 0.7112
Epoch 6/20
150/150 [==============================] - 1s 5ms/step - loss: 0.6012 - accuracy: 0.7112
Epoch 7/20
150/150 [==============================] - 1s 5ms/step - loss: 0.6012 - accuracy: 0.7112
Epoch 8/20
150/150 [==============================] - 1s 5ms/step - loss: 0.6011 - accuracy: 0.7112
Epoch 9/20
150/150 [==============================] - 1s 5ms/step - loss: 0.6012 - accuracy: 0.7112
Epoch 10/20
150/150 [==============================] - 1s 5ms/step - loss: 0.6011 - accuracy: 0.7112

In [100]:
_,acc = keras.evaluate(x_train,y_train)
print(acc*100)

118/118 [==============================] - 1s 3ms/step - loss: 0.6011 - accuracy: 0.7112
71.11999988555908


In [101]:
y_pred = keras.predict(x_test)
y_pred = (y_pred>0.5)

40/40 [==============================] - 0s 2ms/step


In [102]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6904

In [103]:
from keras import backend as K
def r_loss(y_test,y_pred):
    res=K.sum(K.square(y_test-y_pred))    
    total=K.sum(K.square(y_test-K.mean(y_test)))
    return 1-(1-res/(total+K.epsilon()))

In [104]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [105]:
X.shape

(5000, 12)

In [106]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [107]:
def cost_function(X, y, W):
    y_hat = sigmoid(np.dot(X, W))
    cost = -np.mean(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))
    return cost

In [125]:
def pso(X, y, n_particles, n_iterations):
    n_features = 12
    lb = -1
    ub = 1
    swarm_size = (n_particles, n_features)
    V = np.random.uniform(low=-1, high=1, size=swarm_size)
    X = np.random.uniform(low=lb, high=ub, size=swarm_size)
    P_best = X.copy()
    P_best_fitness = np.zeros(n_particles)
    for i in range(n_particles):
        P_best_fitness[i] = cost_function(X[i], y, X[i])
    G_best = X[np.argmin(P_best_fitness)].copy()
    G_best_fitness = np.min(P_best_fitness)
    for iteration in range(n_iterations):
        w = 0.5
        c1 = 0.8
        c2 = 0.9
        r1 = np.random.uniform(size=swarm_size)
        r2 = np.random.uniform(size=swarm_size)
        V = w*V + c1*r1*(P_best - X) + c2*r2*(G_best - X)
        X = X + V
        X[X < lb] = lb
        X[X > ub] = ub
        for i in range(n_particles):
            fitness = cost_function(X[i], y, X[i])
            if fitness < P_best_fitness[i]:
                P_best[i] = X[i].copy()
                P_best_fitness[i] = fitness
        if np.min(P_best_fitness) < G_best_fitness:
            G_best = X[np.argmin(P_best_fitness)].copy()
            G_best_fitness = np.min(P_best_fitness)
    return G_best

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = (X_train - np.mean(X_train, axis=0))/np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_train, axis=0))/np.std(X_train, axis=0)
n_input = 12
n_hidden = 2
n_output = 1

In [127]:

W1 = pso(X_train, y_train, n_particles=20, n_iterations=50)
W2 = pso(sigmoid(np.dot(X_train, W1)), y_train, n_particles=20, n_iterations=50)
W = [W1, W2]


In [129]:
X_train.shape,W[0],W[1]


((4000, 12),
 array([-0.01322888,  0.00299523,  0.02417005, -0.01042317,  0.00046687,
         0.0150289 , -0.0109756 ,  0.02387997, -0.05350616, -0.0136264 ,
        -0.06431793, -0.00192305]),
 array([-0.00894096,  0.02994397, -0.05586617,  0.01218375, -0.01265208,
         0.0397728 , -0.01048481, -0.0093918 , -0.00340826, -0.00142552,
        -0.11931609, -0.01469532]))

In [130]:
W[0]=W[0].reshape(12,1)
W[1]=W[1].reshape(12,1)
y_pred = sigmoid(X_train)
y_pred[:4]

array([[0.24729109, 0.24298409, 0.69122923, 0.21324632, 0.23031344,
        0.74427953, 0.25953973, 0.36531917, 0.42027764, 0.94748412,
        0.43717244, 0.22743823],
       [0.24729109, 0.35144422, 0.46793384, 0.36628912, 0.6317787 ,
        0.52411117, 0.25953973, 0.36531917, 0.42027764, 0.94748412,
        0.43717244, 0.6937634 ],
       [0.24729109, 0.24298409, 0.89595081, 0.27686039, 0.23031344,
        0.91100523, 0.25953973, 0.36531917, 0.42027764, 0.41442592,
        0.43717244, 0.6937634 ],
       [0.59254796, 0.60699199, 0.22884372, 0.49377613, 0.23031344,
        0.2941542 , 0.79154758, 0.36531917, 0.42027764, 0.41442592,
        0.43717244, 0.6937634 ]])

In [131]:
y_test[:4]

array([1, 1, 0, 0], dtype=int64)